# Census and GeoJSON Data EDA

In [1]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
from requests_html import HTMLSession
import json

import itertools

import re

from time import time
from datetime import datetime, timedelta

from shapely.geometry import Polygon

# 1. import census data from `census.gov`

Population data taken from [census.gov](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/).

Since 2020 Census data have not been released yet, we will use 2019 population estimates.

Looking at the [data dictionary](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.pdf), we only want the names and FIPS columns (eg. `STATE`, `STNAME`) and `POPESTIMATE2019`.

In [2]:
with urlopen('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv') as response:
    pop_df = pd.read_csv(
        response, 
        usecols=['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'POPESTIMATE2019'], 
        encoding='latin-1',        # to avoid unicode error
        dtype={'STATE':'str',
               'COUNTY':'str'}
    )
pop_df.head()

,STATE,COUNTY,STNAME,CTYNAME,POPESTIMATE2019
0,01,000,Alabama,Alabama,4903185
1,01,001,Alabama,Autauga County,55869
2,01,003,Alabama,Baldwin County,223234
3,01,005,Alabama,Barbour County,24686
4,01,007,Alabama,Bibb County,22394


In [3]:
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3193 entries, 0 to 3192
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   STATE            3193 non-null   object
 1   COUNTY           3193 non-null   object
 2   STNAME           3193 non-null   object
 3   CTYNAME          3193 non-null   object
 4   POPESTIMATE2019  3193 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 124.9+ KB


Notice that county names provided by the US census contain descriptive terms, such as 'County', whereas the NYTimes data does not.

In [4]:
def optimize(df):
    '''
    Optimizes the data types in a pandas dataframe.
    '''
    dft = df.copy()
    # converts to datetime if possible
    dft = dft.apply(lambda col: pd.to_datetime(col, errors='ignore') if col.dtypes=='object' else col)
    # if there are less than half as many unique values as there are rows, convert to category
    for col in dft.select_dtypes(include='object'):
        if len(dft[col].unique()) / len(df[col]) < 0.5:
            dft[col] = dft[col].astype('category')
    # downcasts numeric columns if possible
    dft = dft.apply(lambda col: pd.to_numeric(col, downcast='integer') if col.dtypes=='int64' else col)
    dft = dft.apply(lambda col: pd.to_numeric(col, downcast='float') if col.dtypes=='float64' else col)
    return dft

In [5]:
# remove state population data
pop_df = pop_df[pop_df['COUNTY'] != '000']

# rename columns to better-match nytimes data (and personal preference)
pop_df.rename(
    columns={
        'STATE':'statefips',
        'COUNTY':'countyfips',
        'STNAME':'state',
        'CTYNAME':'county',
        'POPESTIMATE2019':'population'
    }, inplace=True
)

# create county fips column
pop_df['fips'] = pop_df['statefips'] + pop_df['countyfips']
pop_df.drop(columns=['statefips', 'countyfips'], inplace=True)

# remove descriptive terms from county names
county_terms = ['County', 'Parish', 'Municipality']
for term in county_terms:
    pop_df['county'] = pop_df['county'].str.replace(' ' + term, '')
    
pop_df = optimize(pop_df)
pop_df.head()

,state,county,population,fips
1,Alabama,Autauga,55869,01001
2,Alabama,Baldwin,223234,01003
3,Alabama,Barbour,24686,01005
4,Alabama,Bibb,22394,01007
5,Alabama,Blount,57826,01009


In [6]:
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 1 to 3192
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   state       3142 non-null   category
 1   county      3142 non-null   object  
 2   population  3142 non-null   int32   
 3   fips        3142 non-null   object  
dtypes: category(1), int32(1), object(2)
memory usage: 91.9+ KB


## check county names against NYTimes data

We eventually need to merge `nyt_df` and `pop_df`, so let's see how they match with each other:

In [7]:
with urlopen('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv') as response:
    nyt_df = optimize(pd.read_csv(
        response,
        dtype={'fips':'str'}
    ))
nyt_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [8]:
county_diffs = list(set(nyt_df['county']) - set(pop_df['county']))
len(county_diffs)

82

In [10]:
county_diffs

['Florida',
 'Isabela',
 'Aguadilla',
 'Patillas',
 'Naguabo',
 'Maunabo',
 'Caguas',
 'Toa Baja',
 'Vega Alta',
 'Adjuntas',
 'Luquillo',
 'Mayaguez',
 'Corozal',
 'Guanica',
 'Maricao',
 'Cidra',
 'Quebradillas',
 'San Lorenzo',
 'Penuelas',
 'Yabucoa',
 'Tinian',
 'Barceloneta',
 'San German',
 'Utuado',
 'Jayuya',
 'Fajardo',
 'Hatillo',
 'New York City',
 'Guayanilla',
 'Culebra',
 'Unknown',
 'Camuy',
 'Santa Isabel',
 'Arecibo',
 'Lajas',
 'Orocovis',
 'Comerio',
 'Salinas',
 'Joplin',
 'Ceiba',
 'Villalba',
 'Hormigueros',
 'Manati',
 'Cayey',
 'Vieques',
 'Coamo',
 'Morovis',
 'Moca',
 'Anasco',
 'Lares',
 'Saipan',
 'San Sebastian',
 'Trujillo Alto',
 'Catano',
 'Dorado',
 'Barranquitas',
 'Arroyo',
 'Gurabo',
 'Ponce',
 'Las Piedras',
 'Canovanas',
 'Guayama',
 'Cabo Rojo',
 'Sabana Grande',
 'Kansas City',
 'Juana Diaz',
 'Carolina',
 'Aguas Buenas',
 'Rincon',
 'Aguada',
 'Ciales',
 'Naranjito',
 'Yauco',
 'Bayamon',
 'Las Marias',
 'Humacao',
 'Vega Baja',
 'Toa Alta',
 '

The census county data is missing all municipios from [Puerto Rico](https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-puerto-rico-municipios.html), so we need to append that data to `pop_df`.

## optional: import Puerto Rico census data

Future work: find a way to incorporate Puerto Rico into the Altair map.

In [14]:
# with urlopen('https://www2.census.gov/programs-surveys/popest/tables/2010-2019/municipios/totals/prm-est2019-annres.xlsx') as response:
#     pr_df = pd.read_excel(response, header=3)
pr_df = pd.read_excel('data/prm-est2019-annres.xlsx', header=3)
pr_df = pr_df[['Unnamed: 0', 2019]]
pr_df.rename(
    columns={
        'Unnamed: 0':'county',
        2019:'population'
    }, inplace=True
)
pr_df = pr_df[~pr_df['population'].isna()]
pr_df['population'] = pr_df['population'].astype('int')
pr_df.head()

,county,population
0,Puerto Rico,3193694
1,".Adjuntas Municipio, Puerto Rico",17363
2,".Aguada Municipio, Puerto Rico",36694
3,".Aguadilla Municipio, Puerto Rico",50265
4,".Aguas Buenas Municipio, Puerto Rico",24814


In [15]:
pr_df['county'] = [s[0] if len(s) > 0 else s for s in pr_df['county'].str.findall("\.([\w\s]+) Municipio\,.+")]
pr_df = pr_df.iloc[1:]          # removing the territory as a whole from the table
pr_df.head()

,county,population
1,Adjuntas,17363
2,Aguada,36694
3,Aguadilla,50265
4,Aguas Buenas,24814
5,Aibonito,22108


We also need to add `fips` codes for all of the municipios.

### import Puerto Rico `fips`

In [16]:
sess = HTMLSession()
res = sess.get('https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county')
table = res.html.find('table.wikitable > tbody > tr')
# puerto rico is fips 72
pr_fips = [[tr.find('td')[1].text, tr.find('td')[0].text] for tr in table[1:] if tr.find('td')[0].text[:2] == '72']
pr_fips_df = pd.DataFrame(pr_fips)
pr_fips_df.rename(
    columns={
        0:'county',
        1:'fips'
    }, inplace=True
)
pr_fips_df.head()

,county,fips
0,Adjuntas Municipality,72001
1,Aguada Municipality,72003
2,Aguadilla Municipality,72005
3,Aguas Buenas Municipality,72007
4,Aibonito Municipality,72009


In [17]:
pr_fips_df['county'] = [s[0] if len(s) > 0 else s for s in pr_fips_df['county'].str.findall("([\w\s]+) Municipality")]
pr_fips_df.head()

,county,fips
0,Adjuntas,72001
1,Aguada,72003
2,Aguadilla,72005
3,Aguas Buenas,72007
4,Aibonito,72009


In [18]:
len(list(set(pr_fips_df['county']) - set(pr_df['county'])))

0

In [19]:
pr_df = pr_df.merge(pr_fips_df, on='county')
pr_df['state'] = 'Puerto Rico'
pr_df.head()

,county,population,fips,state
0,Adjuntas,17363,72001,Puerto Rico
1,Aguada,36694,72003,Puerto Rico
2,Aguadilla,50265,72005,Puerto Rico
3,Aguas Buenas,24814,72007,Puerto Rico
4,Aibonito,22108,72009,Puerto Rico


In [20]:
pop_df = pop_df.append(pr_df, ignore_index=True)
pop_df.tail()

,state,county,population,fips
3215,Puerto Rico,Vega Baja,50023,72145
3216,Puerto Rico,Vieques,8386,72147
3217,Puerto Rico,Villalba,21372,72149
3218,Puerto Rico,Yabucoa,32282,72151
3219,Puerto Rico,Yauco,33575,72153


In [21]:
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state       3220 non-null   object
 1   county      3220 non-null   object
 2   population  3220 non-null   int32 
 3   fips        3220 non-null   object
dtypes: int32(1), object(3)
memory usage: 88.2+ KB


## check county names against NYTimes data (again)

In [22]:
county_diffs = list(set(nyt_df['county']) - set(pop_df['county']))
len(county_diffs)

21

In [23]:
county_diffs

['Kansas City',
 'New York City',
 'Juana Diaz',
 'Rincon',
 'Unknown',
 'Anasco',
 'Mayaguez',
 'Guanica',
 'Saipan',
 'San Sebastian',
 'Bayamon',
 'Las Marias',
 'Comerio',
 'Catano',
 'Joplin',
 'Penuelas',
 'Tinian',
 'San German',
 'Canovanas',
 'Manati',
 'Loiza']

The NYTimes dataset is missing diacritical marks in their names. While it would be easier to replace diacritical marks with their "standard" character counterparts, we will preserve them in our final dataframe in the interest of cultural accuracy. This will be handled when we merge `pop_df` with `nyt_df` in the other notebook.

Since the NYTimes dataset treats `New York City`, `Kansas City`, and `Joplin` as their own entities, we need to add them to `pop_df` in addition to adding the census data for these three cities. Additional information taken from [census.gov quickfacts]('https://www.census.gov/quickfacts').

We'll use `'nyc'`, `'kc'`, and `'jm'` as our `fips` for these three cities.

In [24]:
pop_df_2 = pd.DataFrame(
    [['New York',
      'New York City',
      8_336_817,
      'nyc'],
     ['Missouri',
      'Kansas City',
      495_327 + 152_960,
      'kc'],
     ['Missouri',
      'Joplin',
      50_925,
      'jm']]
    , columns=pop_df.columns)
pop_df_2

,state,county,population,fips
0,New York,New York City,8336817,nyc
1,Missouri,Kansas City,648287,kc
2,Missouri,Joplin,50925,jm


In [25]:
pop_df = optimize(pop_df.append(pop_df_2, ignore_index=True))
pop_df[pop_df['fips'] == 'nyc']

,state,county,population,fips
3220,New York,New York City,8336817,nyc


## save results to csv

In [26]:
pop_df.to_csv('data/pop_df.csv', index=False)

# 2. import geojson for boundaries and census areas

In [27]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    county_json = json.load(response)

In [28]:
county_json['features'][0]

{'type': 'Feature',
 'properties': {'GEO_ID': '0500000US01001',
  'STATE': '01',
  'COUNTY': '001',
  'NAME': 'Autauga',
  'LSAD': 'County',
  'CENSUSAREA': 594.436},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-86.496774, 32.344437],
    [-86.717897, 32.402814],
    [-86.814912, 32.340803],
    [-86.890581, 32.502974],
    [-86.917595, 32.664169],
    [-86.71339, 32.661732],
    [-86.714219, 32.705694],
    [-86.413116, 32.707386],
    [-86.411172, 32.409937],
    [-86.496774, 32.344437]]]},
 'id': '01001'}

In [29]:
fips_to_add_to_json = list(set(nyt_df['fips']) - set([f['id'] for f in county_json['features']]))
fips_to_add_to_json

[nan, '46102', '02158', '69110', '69120']

The `plotly` county GeoJSON dataset is missing Kusilvak Census Area (`'02158'`) and Oglala Lakota County(`'46102'`), in addition to the three cities included in the NYTimes data (New York City, Kansas City, Joplin). GeoJSON data for these five areas compiled from [nomanatim](https://nominatim.openstreetmap.org/) and [polygons](http://polygons.openstreetmap.fr/).

- Search for the area at [nomanatim](https://nominatim.openstreetmap.org/).
- Select `details` from the relevant entry.
- Copy the numeric `code` under `OSM`, ignoring "relation". Eg. for New York City, copy `175905`.
- Search for the `code` at [polygons](http://polygons.openstreetmap.fr/).
- For our purposes, GeoJSONs were selected according to the following criteria: (1) sparsity of vertices (`NPoints`) and (2) accuracy of shape.

In [30]:
# new york city, ny
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/nyc.txt') as response:
    nyc_json = json.load(response)

# kansas city, mo/ks
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/kcm.txt') as response:
    kcm_json = json.load(response)
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/kck.txt') as response:
    kck_json = json.load(response)
kc_json = dict(
    coordinates = kcm_json['coordinates'] + kck_json['coordinates']
)

# joplin, mo
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/jm.txt') as response:
    jm_json = json.load(response)

# oglala lakota county, nd
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/olsd.txt') as response:
    olsd_json = json.load(response)

# kusilvak census area, ak
with urlopen('https://raw.githubusercontent.com/jydiw/nyt-covid-19-data/master/data/kca.txt') as response:
    kca_json = json.load(response)

In [31]:
# https://stackoverflow.com/questions/41271146/
def clean_coordinates(c):
    return [list(itertools.chain(*d)) for d in c]

In [32]:
add_to_json_dict = {
    '02158':{'area':17_081.43,
             'name':'Kusilvak Census Area',
             'coordinates':clean_coordinates(kca_json['coordinates'])}, 
    '46102':{'area':2_093.90,
             'name':'Oglala Lakota',
             'coordinates':clean_coordinates(olsd_json['coordinates'])},
    'jm':{'area':35.56,
          'name':'Joplin',
          'coordinates':clean_coordinates(jm_json['geometries'][0]['coordinates'])},
    'kc':{'area':124.81+314.95,
          'name':'Kansas City',
          'coordinates':clean_coordinates(kc_json['coordinates'])},
    'nyc':{'area':302.64,
           'name':'New York City',
           'coordinates':clean_coordinates(nyc_json['coordinates'])}
}

In [40]:
for fips in ['02158', '46102', 'jm', 'kc', 'nyc']:
    county_json['features'].append(
        {
            'geometry': {'coordinates': add_to_json_dict[fips]['coordinates'],
                         'type': 'Polygon'},
            'id': fips,
            'properties': {'NAME': add_to_json_dict[fips]['name'],
                           'CENSUSAREA': add_to_json_dict[fips]['area']},
            'type': 'Feature'
        }
    )

In [41]:
with open('data/county_json.json', 'w') as f:
    json.dump(county_json, f)

## add centroid latitude and longitude coordinates and county area to `pop_df`

In [42]:
def centroid(i, j=county_json):
    for d in j['features']:
        if d['id'] == i:
            shapes = np.array(d['geometry']['coordinates']).flatten()
            try:
                areas = [Polygon(shape).area for shape in shapes]
                p = Polygon(shapes[areas.index(max(areas))])
                lon, lat = p.centroid.coords[0]
            except:
                shapes = np.reshape(shapes, (-1, 2))
                p = Polygon(shapes)
                lon, lat = p.centroid.coords[0]
            return lon, lat

In [43]:
def county_area(i, j=county_json):
    for d in j['features']:
        if d['id'] == i:
            return d['properties']['CENSUSAREA']

In [44]:
tick = time()
pop_df['area'] = pop_df['fips'].apply(county_area)
pop_df['lon'], pop_df['lat'] = zip(*pop_df['fips'].apply(centroid).to_list())
pop_df = optimize(pop_df)
tock = time()
print(tock - tick)

0.7749290466308594


In [45]:
pop_df.head()

,state,county,population,fips,area,lon,lat
0,Alabama,Autauga,55869,01001,594.435974,-86.641197,32.536152
1,Alabama,Baldwin,223234,01003,1589.784058,-87.723953,30.725863
2,Alabama,Barbour,24686,01005,884.875977,-85.389244,31.867889
3,Alabama,Bibb,22394,01007,622.581970,-87.124962,32.996456
4,Alabama,Blount,57826,01009,644.776001,-86.569756,33.985249


## engineer population density column

In [46]:
pop_df['pop_per_area'] = pop_df['population'] / pop_df['area']
pop_df.head()

,state,county,population,fips,area,lon,lat,pop_per_area
0,Alabama,Autauga,55869,01001,594.435974,-86.641197,32.536152,93.986573
1,Alabama,Baldwin,223234,01003,1589.784058,-87.723953,30.725863,140.417813
2,Alabama,Barbour,24686,01005,884.875977,-85.389244,31.867889,27.897695
3,Alabama,Bibb,22394,01007,622.581970,-87.124962,32.996456,35.969561
4,Alabama,Blount,57826,01009,644.776001,-86.569756,33.985249,89.683859


In [47]:
pop_df.to_csv('data/pop_df.csv', index=False)

# put in nytimes notebook

In [ ]:
nyt_df = nyt_df.merge(pop_df, on='fips', suffixes=('_x','')).drop(['county_x', 'state_x'], axis=1)
nyt_df[nyt_df['state'] == 'Puerto Rico']

In [ ]:
nyt_df = nyt_df[nyt_df['county'] != 'Unknown']
list(set(nyt_df['county']) - set(pop_df['county']))

We need to add the population data for these three cities. Additional information taken from [census.gov quickfacts]('https://www.census.gov/quickfacts').

We'll use `'nyc'`, `'kc'`, and `'jm'` as our `fips` for these three cities.